In [1]:
import os, math, copy
import numpy as np
import pandas as pd
from datetime import datetime
from pprint import pprint
from pathlib import Path
import omero
import json

import configparser
import ezomero
from ezomero import rois
#   https://thejacksonlaboratory.github.io/ezomero/ezomero.html#module-ezomero

In [2]:
# Settings for OMERO
home = str(Path.home())
config = configparser.RawConfigParser()
config.read(r'{}\Documents\myKeyVariables.cfg'.format(home))   
envVars = dict(config.items('OMERO'))

# Settings for OMERO
user = envVars['user']
password = envVars['pass']
host = envVars['url']
port = 4064 #4080
secure = True #False

conn = ezomero.connect(user, password, 'MarkovicLab', host=host, port=port, secure=secure)
ezomero.print_projects(conn)

Projects:
	BMS_Teir2_2022:	102
	BMS_Tier1_2021:	201
	3D Analysis MxIF:	915
	IHC_v_MxIF_Developments:	306
	MelPanel4:	311
	BMS_CoreBiopsies_2023:	405
	IHC_v_MxIF_For_March2023_PathReview:	475
	Tassos_updated_2023:	457
	CosMx to CellDive Alignments:	606
	MISCELLANEOUS:	652
	IHC_v_MxIF_June6_PathReview:	712
	Elizabeth's Placenta Project:	751
	Lung Project :	752
	GM_CSF_Pilot:	807
	IHC_v_MxIF_Sept2023_PathReview:	713
	IHCs for CD36 and FASN:	865
	Mackenzie Study and RO1 Practice:	857
	20230620_IHC_MxIF_Dev_CD19_MPO_NRP2:	853
	BMS_Aim2_Pratice:	1005
	BMS_Aim2_Batch1:	1264
	BMS_Aim2_Batch1_Gastric:	1269
	BMS_2024_Mel_GI_H&E:	1274
	Pregnancy_Cancer:	1276


In [3]:
ezomero.print_datasets(conn, project=1264)


Datasets in Project "BMS_Aim2_Batch1":
	BAD QUALITY:	2115
	Melanoma:	2048


In [4]:
allSamps = ezomero.get_image_ids(conn, dataset=2048)
getImageByName = {}
for im in allSamps:
    imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
    imName = imgDesc.getName().replace(".ome.tiff","")
    getImageByName[imName] = im

print("There are {} images to pull.\n[{}]".format(
    len(allSamps), ', '.join(getImageByName.keys() )))

There are 18 images to pull.
[SLIDE-3000_FullPanel, SLIDE-3001_FullPanel, SLIDE-3002_FullPanel, SLIDE-3005_FullPanel, SLIDE-3006_FullPanel, SLIDE-3018_FullPanel, SLIDE-3007_FullPanel, SLIDE-3009_FullPanel, SLIDE-3010_FullPanel, SLIDE-3011_FullPanel, SLIDE-3015_FullPanel, SLIDE-3016_FullPanel, SLIDE-3019_FullPanel, SLIDE-3020_FullPanel, SLIDE-3021_FullPanel, SLIDE-3003_FullPanel, SLIDE-3001a_FullPanel, SLIDE-3001b_FullPanel]


In [5]:
json_annotated_fname = r'{}\Downloads\{}_OMERO_ROIs.geojson'
for im in allSamps:
    imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
    imgPointsIDs = ezomero.get_roi_ids(conn, im)
    imName = imgDesc.getName().replace('\\.ome\\.tiff','')
    print("{} has {} ROIs".format(imName, len(imgPointsIDs)))

    allThesePoints = []
    for i in imgPointsIDs:
        for ppI in ezomero.get_shape_ids(conn, i):
            (polyObj, fill, border, width) = ezomero.get_shape(conn, ppI)
            
            ## Must be Polyline...ignore other types
            if type(polyObj).__name__ == "Polyline":
                packedColor = int('%02x%02x%02x' % (border[0], border[1], border[2]), 16)
                inTb1 = {"name": polyObj.label, "colorRGB": packedColor}
                inTb2 = {"isLocked": False, "classification": inTb1}
                allCoordinates = polyObj.points

                # Make sure the polygon is closed by adding the first point at the end
                firstPoint = allCoordinates[0]
                newCoords = copy.deepcopy(allCoordinates)
                newCoords.append(firstPoint)

                # Create the GeoJSON Polygon geometry
                gsonPoly = {"type": "Polygon", "coordinates": [newCoords]}

                # Create a GeoJSON Feature for this ROI
                newPolyRoi = {
                    "type": "Feature",
                    "properties": inTb2,
                    "geometry": gsonPoly
                }
                allThesePoints.append(newPolyRoi)
            else:
                print("Skip: {} [{}]".format(type(polyObj).__name__, ppI))
    
    # Create the GeoJSON FeatureCollection
    geojson_data = {
        "type": "FeatureCollection",
        "features": allThesePoints
    }
    
    # Write to the output file in GeoJSON format
    with open(json_annotated_fname.format(home, imName), "w") as outfile:
        json.dump(geojson_data, outfile, indent=2)

    print(f"GeoJSON file saved to {json_annotated_fname.format(home, imName)}")

SLIDE-3000_FullPanel.ome.tiff has 9 ROIs
GeoJSON file saved to C:\Users\m088378\Downloads\SLIDE-3000_FullPanel.ome.tiff_OMERO_ROIs.geojson
SLIDE-3001_FullPanel.ome.tiff has 1 ROIs
GeoJSON file saved to C:\Users\m088378\Downloads\SLIDE-3001_FullPanel.ome.tiff_OMERO_ROIs.geojson
SLIDE-3002_FullPanel.ome.tiff has 2 ROIs
GeoJSON file saved to C:\Users\m088378\Downloads\SLIDE-3002_FullPanel.ome.tiff_OMERO_ROIs.geojson
SLIDE-3005_FullPanel.ome.tiff has 2 ROIs
GeoJSON file saved to C:\Users\m088378\Downloads\SLIDE-3005_FullPanel.ome.tiff_OMERO_ROIs.geojson
SLIDE-3006_FullPanel.ome.tiff has 4 ROIs
GeoJSON file saved to C:\Users\m088378\Downloads\SLIDE-3006_FullPanel.ome.tiff_OMERO_ROIs.geojson
SLIDE-3018_FullPanel.ome.tiff has 0 ROIs
GeoJSON file saved to C:\Users\m088378\Downloads\SLIDE-3018_FullPanel.ome.tiff_OMERO_ROIs.geojson
SLIDE-3007_FullPanel.ome.tiff has 4 ROIs
GeoJSON file saved to C:\Users\m088378\Downloads\SLIDE-3007_FullPanel.ome.tiff_OMERO_ROIs.geojson
SLIDE-3009_FullPanel.ome.ti

Skip: Point [121654]
Skip: Point [121655]
Skip: Point [121656]
Skip: Point [121657]
Skip: Point [121658]
Skip: Point [121660]
Skip: Point [121661]
Skip: Point [121662]
Skip: Point [121663]
Skip: Point [121664]
Skip: Point [121665]
Skip: Point [121666]
Skip: Point [121667]
Skip: Point [121668]
Skip: Point [121669]
Skip: Point [121670]
Skip: Point [121671]
Skip: Point [121672]
Skip: Point [121673]
Skip: Point [121674]
Skip: Point [121675]
Skip: Point [121676]
Skip: Point [121677]
Skip: Point [121678]
Skip: Point [121679]
Skip: Point [121680]
Skip: Point [121681]
Skip: Point [121682]
Skip: Point [121683]
Skip: Point [121684]
Skip: Point [121685]
Skip: Point [121686]
Skip: Point [121687]
Skip: Point [121688]
Skip: Point [121689]
Skip: Point [121690]
Skip: Point [121691]
Skip: Point [121692]
Skip: Point [121693]
Skip: Point [121694]
Skip: Point [121695]
Skip: Point [121696]
Skip: Point [121697]
Skip: Point [121698]
Skip: Point [121699]
Skip: Point [121700]
Skip: Point [121701]
Skip: Point [

Skip: Point [121117]
Skip: Point [121118]
Skip: Point [121119]
Skip: Point [121120]
Skip: Point [121121]
Skip: Point [121122]
Skip: Point [121123]
Skip: Point [121124]
Skip: Point [121125]
Skip: Point [121126]
Skip: Point [121127]
Skip: Point [121128]
Skip: Point [121130]
Skip: Point [121131]
Skip: Point [121132]
Skip: Point [121133]
Skip: Point [121134]
Skip: Point [121135]
Skip: Point [121136]
Skip: Point [121137]
Skip: Point [121138]
Skip: Point [121139]
Skip: Point [121140]
Skip: Point [121141]
Skip: Point [121142]
Skip: Point [121143]
Skip: Point [121144]
Skip: Point [121145]
Skip: Point [121146]
Skip: Point [121147]
Skip: Point [121148]
Skip: Point [121149]
Skip: Point [121150]
Skip: Point [121151]
Skip: Point [121152]
Skip: Point [121153]
Skip: Point [121154]
Skip: Point [121155]
Skip: Point [121156]
Skip: Point [121157]
Skip: Point [121158]
Skip: Point [121159]
Skip: Point [121160]
Skip: Point [121161]
Skip: Point [121162]
Skip: Point [121163]
Skip: Point [121164]
Skip: Point [